In [1]:
import torch

In [3]:
import lightning.pytorch as pl

In [4]:
import pandas as pd

In [6]:
import os

In [7]:
os.getcwd()

'/Users/nikolaushouben/Desktop/net-load-forecasting/notebooks'

In [17]:
df_netload = pd.read_hdf('../data/data_net_load_forecasting.h5', key='1min/netload')

df_load = pd.read_hdf('../data/data_net_load_forecasting.h5', key='1min/community_load')

df_pv = pd.read_hdf('../data/data_net_load_forecasting.h5', key='1min/community_pv')

In [20]:
df_netload

component,net_community_load
time,
2014-08-27 00:00:00,3248.441755
2014-08-27 00:01:00,3167.381976
2014-08-27 00:02:00,3152.603824
2014-08-27 00:03:00,3188.512354
2014-08-27 00:04:00,3156.355887
...,...
2016-12-29 23:55:00,3218.672501
2016-12-29 23:56:00,3292.832497
2016-12-29 23:57:00,3331.624979


In [33]:
df_pv[:5]

component,community_pv
time,
2014-08-27 00:00:00,0.0
2014-08-27 00:01:00,0.0
2014-08-27 00:02:00,0.0
2014-08-27 00:03:00,0.0
2014-08-27 00:04:00,0.0


In [136]:
from darts.utils.data.sequential_dataset import MixedCovariatesSequentialDataset


from darts import TimeSeries

In [100]:
ts_netload = TimeSeries.from_dataframe(df_netload)
ts_pv = TimeSeries.from_dataframe(df_pv)

In [69]:
from darts.dataprocessing.encoders.encoders import PastDatetimeAttributeEncoder, SequentialEncoder

In [123]:
input_chunk_length = 3
output_chunk_length = 5

add_encoder = {'cyclic': {'past': ['day']}}
enc2 = SequentialEncoder(add_encoder, input_chunk_length = input_chunk_length , output_chunk_length = output_chunk_length, takes_future_covariates = True, takes_past_covariates = True)

In [128]:
past_cov, _  = enc2.encode_train(ts_netload)

In [146]:
past_cov

<TimeSeries (DataArray) (time: 1232635, component: 2, sample: 1)>
array([[[-0.72479279],
        [ 0.68896692]],

       [[-0.72479279],
        [ 0.68896692]],

       [[-0.72479279],
        [ 0.68896692]],

       ...,

       [[-0.39435586],
        [ 0.91895781]],

       [[-0.39435586],
        [ 0.91895781]],

       [[-0.39435586],
        [ 0.91895781]]])
Coordinates:
  * time       (time) datetime64[ns] 2014-08-27 ... 2016-12-29T23:54:00
  * component  (component) object 'darts_enc_pc_cyc_day_sin' 'darts_enc_pc_cy...
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [137]:
ds = MixedCovariatesSequentialDataset(target_series=ts_netload,
            past_covariates=past_cov,
            future_covariates=ts_pv,
            input_chunk_length=input_chunk_length,
            output_chunk_length=output_chunk_length,
            max_samples_per_ts=None)

(array([[3278.26488631],
        [3303.07417107],
        [3631.54666583]]),
 array([[-0.39435586,  0.91895781],
        [-0.39435586,  0.91895781],
        [-0.39435586,  0.91895781]]),
 array([[0.],
        [0.],
        [0.]]),
 array([[0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 None,
 array([[3218.67250125],
        [3292.83249728],
        [3331.6249787 ],
        [3303.0241917 ],
        [3466.85499334]]))

In [107]:
from torch.utils.data import DataLoader

In [148]:
from typing import List, Tuple

def _batch_collate_fn(batch: List[Tuple]) -> Tuple:
    """
    Returns a batch Tuple from a list of samples
    """
    aggregated = []
    first_sample = batch[0]
    for i in range(len(first_sample)):
        elem = first_sample[i]
        if isinstance(elem, np.ndarray):
            aggregated.append(
                torch.from_numpy(np.stack([sample[i] for sample in batch], axis=0))
            )
        elif elem is None:
            aggregated.append(None)
        elif isinstance(elem, TimeSeries):
            aggregated.append([sample[i] for sample in batch])
    return tuple(aggregated)

In [150]:
train_loader = DataLoader(
    ds,
    batch_size=32,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    drop_last=False,
    collate_fn=_batch_collate_fn,
)

In [56]:
from darts.models import BlockRNNModel

The `LightGBM` module could not be imported. To enable LightGBM support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
The `Prophet` module could not be imported. To enable Prophet support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
The `CatBoost` module could not be imported. To enable CatBoost support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
/Users/nikolaushouben/opt/anaconda3/envs/netloadpaper/lib/python3.9/site-packages/statsforecast/core.py:25: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
The StatsForecast module could not be imported. To enable support for the StatsForecastAutoARIMA, StatsForecastAutoETS and Cr

In [58]:
m = BlockRNNModel(3, 5)

In [60]:
m.fit(ts_netload)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | RNN              | 700   
4 | fc            | Sequential       | 130   
---------------------------------------------------
830       Trainable params
0         Non-trainable params
830       Total params
0.003     Total estimated model params size (MB)


Epoch 0:   3%|▎         | 1081/38520 [00:08<05:10, 120.62it/s, train_loss=nan.0]

/Users/nikolaushouben/opt/anaconda3/envs/netloadpaper/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


BlockRNNModel(input_chunk_length=3, output_chunk_length=5, model=RNN, hidden_dim=25, n_rnn_layers=1, hidden_fc_sizes=None, dropout=0.0)